<h1><span style="color:#0096FF">Women Lens</span></h1>

<h2>Abstract</h2>

The research underlines the role of photography in enhancing women's emancipation. The investigated sub-topics are the timespan perspective and how women photographers were distributed among the decades; the geographical perspective and in which countries women photographers were more active; the professional perspective and how women photographers' parallel occupations changed during the time and around different countries.

<h4>Photographers selection</h4>


Let’s start with a simple and preliminary search. How many women photographers are there in the world? We will use Wikidata, an open knowledge base that provides extensive information about the entities it contains.


### Querying Wikidata with SPARQL

    SELECT ?item ?itemLabel WHERE {
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,es,fr,de,nl,it". }
    ?item wdt:P21 wd:Q6581072;
    wdt:P31 wd:Q5.
    ?item wdt:P106 wd:Q33231.

Then it follows a manual cleaning of the dataset by selecting the photographers of the publication "Donne Fotografe" in the series Foto Note by Contrasto Books. Our final csv dataset counts a total of 188 photographers out of 9821 Wikidata results.

### Manipulating the results: creating data structures and declaration of global variables

Let's manipulate our results creating data structures and variables. We create an array (photographer data) containing all our elements.

In [5]:
%%js

// File variables.js

// Dataset photographer
var dataSet = new Array(); // Variabile principale di tutti i dati
const debug = false; // Variabile di debug
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

// Chart: occupations over the decades
var occupationsChart; // Chart HTML

var minDecade = 2020; // Variabile per decades min
var maxDecade = 0; // Variabile per decades max
var decadesOccupations = {}; // Dataset per chart decades occupations
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

// Occupations divided by groups (array)
const occupationsGroups = {
    architectureSculpture : [
        'architect', 
        'architectural photographer', 
        'urban planner', 
        'sculptor',
        'industrial designer'
    ],
    visualArt : [
        'illustrator', 
        'painter', 
        'drawer', 
        'designer', 
        'graphic designer', 
        'graphic artist', 
        'visual artist', 
        'collagist', 
        'installation artist', 
        'conceptual artist',
        'textile artist',
        'glass artist',
        'fashion designer'
    ],
    literature : [
        'writer', 
        'poet', 
        'literary critic', 
        'novelist', 
        'autobiographer', 
        'diarist', 
        'biographer'
    ],
    cinemaMusicTheatre : [
        'film actress', 
        'cinematographer', 
        'camera operator', 
        'filmmaker', 
        'film director', 
        'director', 
        'film editor', 
        'screenwriter', 
        'film producer', 
        'actress', 
        'stage actress', 
        'choreographer', 
        'ballet dancer', 
        'singer'
    ],
    mediaArt : [
        'photojournalist',
        'war photographer',
        'portrait photographer',
        'fashion photographer',
        'television producer',
        'multi-media artist',
        'music video director',
        'installation artist',
        'conceptual artist'
    ],
    nonArtistic : [
        'botanist',
        'model',
        'journalist',
        'activist',
        'teacher',
        'hatter',
        'oceanographer',
        'wet nurse',
        'resistance fighter',
        'anthropologist',
        'feminist',
        'business woman',
        'politician',
        'editor',
        'spy',
        'printmaker',
        'translator'
    ]
};
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

// Full list of occupations
var allOccupations = [
    'writer', 
    'illustrator', 
    'botanical collector', 
    'botanist', 
    'courtesan', 
    'artist', 
    'model', 
    'painter', 
    'photojournalist', 
    'journalist', 
    'architectural photographer', 
    'war photographer', 
    'portrait photographer', 
    'fashion photographer', 
    'actress', 
    'stage actress', 
    'film actress', 
    'activist', 
    'teacher', 
    'opinion journalist', 
    'architect', 
    'urban planner', 
    'drawer', 
    'university teacher', 
    'designer', 
    'resistance fighter', 
    'art dealer', 
    'graphic designer', 
    'illustartor', 
    'graphic artist', 
    'textile artist', 
    'sculptor', 
    'industrial designer', 
    'spy', 
    'literary critic', 
    'novelist', 
    'autobiographer', 
    'poet', 
    'choreographer', 
    'documentarian', 
    'war correspondent', 
    'glass artist', 
    'printmaker', 
    'translator', 
    'cinematographer', 
    'camera operator', 
    'filmmaker', 
    'editor', 
    'film director', 
    'director', 
    'fashion designer', 
    'film editor', 
    'singer', 
    'hatter', 
    'screenwriter', 
    'oceanographer', 
    'diarist', 
    'wet nurse', 
    'French Resistance fighter', 
    'anthropologist', 
    'feminist', 
    'ballet dancer', 
    'visual artist', 
    'business woman', 
    'politician', 
    'publisher', 
    'fashion editor', 
    'collagist', 
    'multi-media artist', 
    'biographer', 
    'installation artist', 
    'conceptual artist', 
    'video artist', 
    'television producer', 
    'exhibition curator', 
    'performance artist', 
    'art theorist', 
    'video installation artist', 
    'music video director', 
    'pedagogue', 
    'film producer', 
    'actor'
];
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

// Chart: countries
var countriesChart; // Chart HTML

var countries = {}; // Dataset per chart
var countriesLabels = []; // Labels per chart
var countriesValues = []; // Valori per chart
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

// Chart: occupations over the countries
var occupationsCountriesChart; // Chart HTML

var countriesOccupations = {} // Dataset per chart

// Lista di countries selezionate per il grafico
const selectedCountries = [
    'United States of America',
    'France',
    'Germany',
    'United Kingdom'
];

// Lista di occupazioni selezionate
const selectedOccupations = [
    'architectural photographer',
    'sculptor',
    'writer',
    'painter',
    'film director',
    'photojournalist',
    'model',
    'war photographer',
    'screenwriter',
    'visual artist',
    'teacher',
    'printmaker',
    'journalist'
];
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

// Chart: cake
var phTypeChart; // Chart HTML
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

// Link API, i dataset vengono recuperati da questi 3 link
const urlTimeLine = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQsRkWcOsiueCplEVwhyWjtVqUHvG94knzINrD5-pgrzcO31_anwLXhDzBN-Lgy9NzqEbzX7ei66X6L/pub?gid=0&single=true&output=csv';
const urlCountry = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQsRkWcOsiueCplEVwhyWjtVqUHvG94knzINrD5-pgrzcO31_anwLXhDzBN-Lgy9NzqEbzX7ei66X6L/pub?gid=1905568726&single=true&output=csv';
const urlOccupations = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQsRkWcOsiueCplEVwhyWjtVqUHvG94knzINrD5-pgrzcO31_anwLXhDzBN-Lgy9NzqEbzX7ei66X6L/pub?gid=1144740092&single=true&output=csv';
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

// Colors (17)
// Colori che abbiamo selezionato per i grafici
const graphBackgroundColor = [
    'rgba(251, 80, 18, 0.6)',
    'rgba(1, 253, 246, 0.6)',
    'rgba(203, 186, 237, 0.6)',
    'rgba(233, 223, 0, 0.6)',
    'rgba(3, 252, 186, 0.6)',
    'rgba(150, 189, 198, 0.6)',
    'rgba(129, 150, 143, 0.6)',
    'rgba(232, 204, 191, 0.6)',
    'rgba(42, 42, 114, 0.6)',
    'rgba(233, 214, 236, 0.6)',
    'rgba(233, 128, 110, 0.6)',
    'rgba(197, 155, 118, 0.6)',
    'rgba(120, 188, 97, 0.6)',
    'rgba(192, 199, 129, 0.6)',
    'rgba(255, 172, 228, 0.6)',
    'rgba(236, 78, 32, 0.6)',
    'rgba(138, 79, 125, 0.6)'
];

const graphBorderColor = [
    'rgba(251, 80, 18, 1)',
    'rgba(1, 253, 246, 1)',
    'rgba(203, 186, 237, 1)',
    'rgba(233, 223, 0, 1)',
    'rgba(3, 252, 186, 1)',
    'rgba(150, 189, 198, 1)',
    'rgba(129, 150, 143, 1)',
    'rgba(232, 204, 191, 1)',
    'rgba(42, 42, 114, 1)',
    'rgba(233, 214, 236, 1)',
    'rgba(233, 128, 110, 1)',
    'rgba(197, 155, 118, 1)',
    'rgba(120, 188, 97, 1)',
    'rgba(192, 199, 129, 1)',
    'rgba(255, 172, 228, 1)',
    'rgba(236, 78, 32, 1)',
    'rgba(138, 79, 125, 1)'
];
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

// Photographer class
// Classe contenente tutti i dati e i metodi utili per la gestione delle fotografe
class photographerData {
    // Inizializzatore dei dati, in caso non vengano passati, vengono inizializzati a valore nullo (String, o array)
    constructor(name = '', country = '', occupations = [], birthDate = '', deathDate = '', phLink = '', cLink = '', birthDecade = '', deathDecade = '', type = '') {
        this.name = name;
        this.country = country;
        this.occupations = occupations;
        this.birthDate = birthDate;
        this.deathDate = deathDate;
        this.phLink = phLink;
        this.cLink = cLink;
        this.birthDecade = birthDecade;
        this.deathDecade = deathDecade;
        this.type = type;
    }

    // Set methods
    // Metodi per settare i dati delle fotografe
    setName(name) {
        this.name = name;
    }

    setCountry(country) {
        this.country = country;
    }

    setOccupations(occupations) {
        this.occupations = occupations;
    }

    setBirthDate(birthDate) {
        this.birthDate = birthDate;
    }

    setDeathDate(deathDate) {
        this.deathDate = deathDate;
    }

    setPhLink(phLink) {
        this.phLink = phLink;
    }

    setCLink(cLink) {
        this.cLink = cLink;
    }

    setBirthDecade(birthDecade) {
        this.birthDecade = birthDecade;
    }

    setDeathDecade(deathDecade) {
        this.deathDecade = deathDecade;
    }

    setType(type) {
        this.type = type;
    }

    // Get methods
    // Metodi per ottenere i dati dalla struttura
    getName() {
        return this.name;
    }

    getCountry() {
        return this.country;
    }

    getOccupations() {
        return this.occupations;
    }

    getBirthDate() {
        return this.birthDate;
    }

    getDeathDate() {
        return this.deathDate;
    }

    getPhLink() {
        return this.phLink;
    }

    getBirthDecade() {
        return this.birthDecade;
    }

    getDeathDecade() {
        return this.deathDecade;
    }

    getType() {
        return this.type;
    }

}

<IPython.core.display.Javascript object>

<h3>1. Timespan: pioneer, revolutionary and visionary photographers</h3>

### Querying Wikidata with SPARQL


The very first query within research topics is about the timespan. Using SPARQL I retrieved the birth date and the death date (when available) of the photographers.


    SELECT ?item ?itemLabel ?birthDate ?birthDateLabel ?deathDate ?deathDateLabel WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,es,fr,de,nl,it". }
      ?item wdt:P21 wd:Q6581072;
        wdt:P31 wd:Q5;
        wdt:P106 wd:Q33231.
      OPTIONAL { ?item wdt:P569 ?birthDate. }
      OPTIONAL { ?item wdt:P570 ?deathDate. }
    }


Then it follows again a manual cleaning of the dataset by selecting only the photographers of the publication and adding "pioneer", "revolutionary" or "visionary" labels depending on the decades of activity when photographers worked. The three labels above match the three volumes of the publication, in which the photographers are divided for decades.

### Data manipulation

In [6]:
%%js
// Inizializzazione dei nomi per la struttura dataset
const initNames = async (csvUrl = './dataset/Country.csv') => {
    // Reset globals
    dataSet = [];

    // Read data
    const data = await (await fetch(csvUrl)).text();

    // Split data csv
    var allRows = data.split(/\r?\n|\r/);

    // Add data to dataSet
    // Scorro allRows riga per riga
    for (let i=1; i<allRows.length; i++) {
        // Divisione di una riga in più argomenti
        var rowCells = allRows[i].split(',');

        // Per comodità segno a cosa corrisponde ogni argomento
        let phLink = rowCells[0];
        let name = rowCells[1];
        let cLink = rowCells[2];
        let country = rowCells[3];

        // Inserisco i dati nella struttura
        let dataToInsert = new photographerData(name);

        dataSet.push(dataToInsert);
    }
}

// Inizializzazione del file csv timeline all'interno della nostra struttura dati
const readTimeline = async (csvUrl = './dataset/Timeline.csv') => { // CSV: item,itemLabel,birthDate,deathDate
    // Read data
    const data = await (await fetch(csvUrl)).text();
    
    // Split data csv
    var allRows = data.split(/\r?\n|\r/);

    // Add data to dataSet
    for (let i=1; i<allRows.length; i++) {
        // Divido per argomento
        var rowCells = allRows[i].split(',');

        // Per comodità segno a cosa corrisponde ogni argomento
        let phLink = rowCells[0];
        let name = rowCells[1];
        let birthDate = rowCells[2];
        let deathDate = rowCells[3];
        let type = rowCells[4];

        // Cerco l'indice corrispondente all'interno del dataset (per nome)
        let objIndex = dataSet.findIndex((ph => ph.name == name));

        // Se l'elemento è presente
        if (objIndex != -1){
            // imposto data di nascita e di morte
            dataSet[objIndex].setBirthDate(birthDate);
            dataSet[objIndex].setDeathDate(deathDate);

            // imposto il tipo
            dataSet[objIndex].setType(type);

            // Divido la data
            let bYear = birthDate.split('/');
            let dYear = deathDate.split('/');

            // Per comodità se la fotografa ha questa data, è ancora viva
            if (deathDate === '01/05/2022'){
                dataSet[objIndex].setDeathDecade(2020);
                maxDecade = 2020;
            }
            else{
                // Altrimenti andiamo a calcolare la decade massima
                dataSet[objIndex].setDeathDecade(parseInt(dYear[2]/10)*10);

                maxDecade = Math.max(maxDecade, dataSet[objIndex].getDeathDecade());
            }
            // Andiamo a calcolare la decade minima
            dataSet[objIndex].setBirthDecade(parseInt(bYear[2]/10)*10);

            minDecade = Math.min(minDecade, dataSet[objIndex].getBirthDecade());
        }
        // Se l'elemento non è presente genera un errore
        else throw new Error('ReadTimeline -> indice non trovato ' + name);
    }
}

<IPython.core.display.Javascript object>

To visualize the results we use the Chart JS library and choose a pie chart including the numbers of pioneer, revolutionary and visionary photographers.

<h4>Conclusions</h4>

Through this first query, we understand that the number of women photographers active during the time is equally spread and quite continuous.

<h3>2. Geographical perspective</h3>

After looking at the historical situation, let us look more closely and focus on the photographers' distribution in the world, by country.

#### Querying Wikidata with SPARQL

    SELECT ?item ?itemLabel ?originCountry ?originCountryLabel WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,es,fr,de,nl,it". }
      ?item wdt:P21 wd:Q6581072;
        wdt:P31 wd:Q5;
        wdt:P106 wd:Q33231.
      OPTIONAL { ?item wdt:P27 ?originCountry. }
    }

Wikidata’s wdt:P27 property specifically refers to the country of citizenship. Since many photographers have and have had multiple nationalities or not always the country of origin coincides with that of citizenship, it is chosen to follow once again the information of the publication going to manually clean up the csv file. The country that appears is therefore the native country of the photographers, which sometimes also coincides with the country of citizenship.

<h4>Data manipulation</h4>

In [7]:
%%js
// Inizializzazione da file csv per countries
const readCountry = async (csvUrl = './dataset/Country.csv') => { // CSV: item,itemLabel,birthCountry,birthCountryLabel
    // Read data
    const data = await (await fetch(csvUrl)).text();

    // Split data csv
    var allRows = data.split(/\r?\n|\r/);

    // Add data to dataSet
    for (let i=1; i<allRows.length; i++) {
        // Divisione per argomento
        var rowCells = allRows[i].split(',');

        // Per comodità segno a cosa corrisponde ogni argomento
        let phLink = rowCells[0];
        let name = rowCells[1];
        let cLink = rowCells[2];
        let country = rowCells[3];

        // Cerchiamo per nome l'elemento
        let objIndex = dataSet.findIndex((ph => ph.name == name));

        // Se l'elemento è presente
        if (objIndex != -1){
            // Imposto country, countryLink e PHLink
            dataSet[objIndex].setCountry(country);
            dataSet[objIndex].setPhLink(phLink);
            dataSet[objIndex].setCLink(cLink);
        }
        // Altrimenti genera un errore
        else throw new Error('ReadCountry -> indice non trovato ' + name)
    }
}

<IPython.core.display.Javascript object>

To visualize the results we use the Chart JS library and choose a bar chart.

<h4>Conclusions</h4>

Through this second query, we understand that the countries more represented are all western: the United States, Germany and France.

<h3>3. Occupations over the decades</h3>

Let’s have a look at the parallel occupations of the photographers among different decades.

#### Querying Wikidata with SPARQL

    SELECT ?item ?itemLabel ?birthDate ?birthDateLabel ?deathDate ?deathDateLabel WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,es,fr,de,nl,it". }
      ?item wdt:P21 wd:Q6581072;
        wdt:P31 wd:Q5;
        wdt:P106 wd:Q33231.
      OPTIONAL { ?item wdt:P569 ?birthDate. }
      OPTIONAL { ?item wdt:P570 ?deathDate. }
    }


    SELECT ?item ?itemLabel ?occupation ?occupationLabel WHERE {
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,es,fr,de,nl,it". }
    ?item wdt:P21 wd:Q6581072;
    wdt:P31 wd:Q5.
    ?item wdt:P106 wd:Q33231.
    OPTIONAL { ?item wdt:P106 ?occupation. }


Here we have combined the first query (about timeline) with a query to retrieve the parallel occupations of the female photographers. The csv of the occupations was built exclusively with Wikidata, as the publication does not provide schematic information about it but rather biographical notes on the life of the photographers.

The generated csv was then manually cleaned up, keeping only the entities of the photographers to which it was actually associated with at least one attribute of parallel occupation in addition to that of the photographer. In the case of multiple parallel occupations, the values have been separated by semicolons within the same cell.


<h4>Data manipulation</h4>

In [8]:
%%js
// Lettura csv per occupazioni
const readOccupations = async (csvUrl = './dataset/Occupations.csv') => { // CSV: item,itemLabel,parallelOccupation,occupationLabel
    // Read data
    const data = await (await fetch(csvUrl)).text();

    // Split data csv
    var allRows = data.split(/\r?\n|\r/);

    // Add data to dataSet
    for (let i=1; i<allRows.length; i++) {
        // Divido in argomenti
        var rowCells = allRows[i].split(',');

        // Per comodità segno a cosa corrisponde ogni argomento
        let phLink = rowCells[0];
        let name = rowCells[1];
        let ocLinks = rowCells[2];
        let occupations = rowCells[3];

        // Cerco l'indice in base al nome
        let objIndex = dataSet.findIndex((ph => ph.name == name));

        // Se presente
        if (objIndex != -1){

            occupations = occupations.split('; ');
            // Inserisco le occupazioni (divise da un ;)
            dataSet[objIndex].setOccupations(occupations);
        }
        // Se non presente, genero un errore
        else throw new Error('ReadOccupations -> indice non trovato ' + name);
    }
}

<IPython.core.display.Javascript object>


To visualize the results we use the Chart JS library and choose a line chart.

<h4>Conclusions</h4>

Through this third query, we mapped all the most relevant parallel occupations of the female photographers over the decades. We are now able to infer some statistical trends.

<h3>4. Occupations over the countries</h3>

Let’s have a look at the parallel occupations of the photographers among different countries.

#### Querying Wikidata with SPARQL

    SELECT ?item ?itemLabel ?birthDate ?birthDateLabel ?deathDate ?deathDateLabel WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,es,fr,de,nl,it". }
      ?item wdt:P21 wd:Q6581072;
        wdt:P31 wd:Q5;
        wdt:P106 wd:Q33231.
      OPTIONAL { ?item wdt:P569 ?birthDate. }
      OPTIONAL { ?item wdt:P570 ?deathDate. }
    }



    SELECT ?item ?itemLabel ?originCountry ?originCountryLabel WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,es,fr,de,nl,it". }
      ?item wdt:P21 wd:Q6581072;
        wdt:P31 wd:Q5;
        wdt:P106 wd:Q33231.
      OPTIONAL { ?item wdt:P27 ?originCountry. }
    }


Here we have combined the second query (about countries) with the query about occupations, in order to see which occupations are more popular in some selected countries.

To visualize the results we use the Chart JS library and choose a bar chart.

<h4>Conclusions</h4>

Through this last query, we mapped all the most relevant parallel occupations of the female photographers among the selected countries. We are now able to infer some statistical trends.
